In [2]:
import csv
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta
from time import strftime
import pandas as pd
import re
import time
from dateutil import rrule
from time import mktime
from requests.exceptions import RetryError
from random import randint

from pytx import ThreatDescriptor
from pytx.vocabulary import ThreatExchange as te
from pytx.access_token import access_token
from pytx.logger import setup_logger
from pytx.vocabulary import PrivacyType as pt

In [ ]:
# GET DATA

# Only for plotting
total_start_date = '2016-01-01'
total_stop_date = '2016-06-21'
#

days = 365 * 2
search_terms = ['abuse', 'phishing', 'malware', 'exploit', 'apt', 'ddos', 'brute', 'scan', 'cve']

access_token(app_id='1898105790402064', app_secret='d8552b29acc35528e64720e89f757151')

search_params = {
    te.FIELDS: ThreatDescriptor._default_fields,
    te.LIMIT: 10000,
    te.SINCE: strftime('%Y-%m-%d %H:%m:%S +0000', (datetime.utcnow() + timedelta(days=(-1*days_back))).timetuple()),
    te.TEXT: search_terms,
    te.UNTIL: strftime('%Y-%m-%d %H:%m:%S +0000', datetime.utcnow().timetuple()),
    te.STRICT_TEXT: False
}
start = datetime.strptime('2016-06-23','%Y-%m-%d')
until = start + timedelta(days=days)
search_params[te.SINCE]='2016-06-22'

def query(dt):
    search_params[te.UNTIL]=dt 
    data_frame = None
    tmp= pd.DataFrame()
    results=None
    for search_term in search_terms:
        print("Searching for '%s' from %s to %s" % (search_term,search_params[te.SINCE], search_params[te.UNTIL]))
        results = ThreatDescriptor.objects(
            fields=search_params[te.FIELDS],
            limit=search_params[te.LIMIT],
            text=search_term, 
            since=search_params[te.SINCE], 
            until=search_params[te.UNTIL],
            strict_text=search_params[te.STRICT_TEXT]
        )
        tmp = pd.DataFrame([result.to_dict() for result in results])
        tmp['search_term'] = search_term

        if data_frame is None:
            data_frame = tmp
        else:
            data_frame = data_frame.append(tmp)

    try:
        # Extract a datetime and timestamp, for easier analysis
        data_frame['ds'] = pd.to_datetime(data_frame.added_on.str[0:10], format='%Y-%m-%d')
        data_frame['ts'] = pd.to_datetime(data_frame.added_on)
        
        # Extract the owner data
        owner = data_frame.pop('owner')
        owner = owner.apply(pd.Series)
        data_frame['name'] = owner.name

        # Extract freeform 'tags' in the description
        def extract_tags(text):
            return re.findall(r'\[([a-zA-Z0-9\:\-\_]+)\]', text)
        data_frame['tags'] = data_frame.description.map(lambda x: [] if x is None else extract_tags(x))

        data_frame.to_csv('fb_'+dt.strftime('%m%d%Y')+'.csv', sep=',')
        
    except AttributeError as e:
        print('Attribute error')
        print(e)

for dt in rrule.rrule(rrule.DAILY, dtstart=start, until=until):  
    while True:
        try:
            query(dt)
        except RetryError:
            print('Retry error')
            time.sleep(randint(100,300))
            continue
        break
    search_params[te.SINCE]=dt

In [8]:
# Concatentate all files togethter in single dataframe
path = Path('')
d = pd.DataFrame()
for f in path.glob('data_per_day/*.csv'):
    cs = pd.read_csv(f, parse_dates=[24, 25])
    cs.drop(list(cs.filter(regex = 'Unnamed')), axis = 1, inplace = True)
    d = pd.concat([d, cs])

print(d.shape)

# Write concatenated files csv
d.to_csv('data.csv', sep=',')

(67049, 25)


In [9]:
# Load data from pre-made data.csv

with open('data.csv') as f:
    d = pd.read_csv(f, parse_dates=[24, 25])
d

['Unnamed: 0' 'added_on' 'confidence' 'description' 'expired_on'
 'first_active' 'id' 'indicator' 'last_active' 'last_updated' 'metadata'
 'my_reactions' 'precision' 'privacy_members' 'privacy_type'
 'raw_indicator' 'review_status' 'severity' 'share_level' 'source_uri'
 'status' 'tags' 'type' 'ds' 'ts' 'name']


/usr/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,added_on,confidence,description,expired_on,first_active,id,indicator,last_active,last_updated,...,review_status,severity,share_level,source_uri,status,tags,type,ds,ts,name
0,0,2016-10-10T04:27:04+0000,50,Teaserinj Malicious Script Injection on Possib...,NaN,NaN,989335801175994,"{'id': '989335794509328', 'indicator': 'http:/...",NaN,2016-10-10T04:27:06+0000,...,UNREVIEWED,SEVERE,GREEN,NaN,MALICIOUS,NaN,URI,2016-10-10,2016-10-10 04:27:04,RiskIQ
1,0,2016-01-01T23:29:55+0000,90,Compromised domain in AS3741. Malicious URL:mo...,NaN,NaN,875325125915484,"{'id': '875325115915485', 'indicator': 'molgri...",NaN,2016-05-21T23:43:18+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:29:55,Cloudmark Threat Exchange
2,1,2016-01-01T23:24:49+0000,90,Compromised domain in AS32613. Malicious URL:p...,NaN,NaN,953897001325079,"{'id': '953896997991746', 'indicator': 'p13855...",NaN,2016-05-26T22:17:16+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:24:49,Cloudmark Threat Exchange
3,2,2016-01-01T23:24:48+0000,90,Compromised domain in AS32613. Malicious URL:h...,NaN,NaN,942876809114340,"{'id': '942876805781007', 'indicator': 'develo...",NaN,2016-05-29T00:08:32+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:24:48,Cloudmark Threat Exchange
4,3,2016-01-01T22:56:19+0000,90,Compromised domain in AS3223. Malicious URL:ww...,NaN,NaN,912399758852107,"{'id': '912399755518774', 'indicator': 'bateri...",NaN,2016-05-28T02:14:54+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:56:19,Cloudmark Threat Exchange
5,4,2016-01-01T22:49:12+0000,90,Compromised domain in AS6724. Malicious URL:ht...,NaN,NaN,930020557087476,"{'id': '930020553754143', 'indicator': 'predif...",NaN,2016-05-29T17:06:50+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:49:12,Cloudmark Threat Exchange
6,5,2016-01-01T22:49:11+0000,90,Compromised domain in AS20013. Malicious URL:i...,NaN,NaN,824054484369537,"{'id': '824054481036204', 'indicator': 'iprote...",NaN,2016-05-31T15:06:12+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:49:11,Cloudmark Threat Exchange
7,6,2016-01-01T22:48:10+0000,90,Compromised domain in AS46606. Malicious URL:h...,NaN,NaN,983145918419311,"{'id': '983145915085978', 'indicator': 'live--...",NaN,2016-05-19T12:49:11+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:48:10,Cloudmark Threat Exchange
8,7,2016-01-01T22:02:25+0000,90,Compromised domain in AS42355. Malicious URL:h...,NaN,NaN,946343195412806,"{'id': '946343192079473', 'indicator': 'kdev.d...",NaN,2016-05-27T10:34:31+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:02:25,Cloudmark Threat Exchange
9,8,2016-01-01T21:45:08+0000,90,Compromised domain in AS48505. Malicious URL:s...,NaN,NaN,1144064518967456,"{'id': '1144064515634123', 'indicator': 'slubn...",NaN,2016-05-26T05:05:23+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 21:45:08,Cloudmark Threat Exchange


In [14]:
d_filtered_compromised = d[d.description.str.startswith('Compromised', na=False)]
d_filtered_compromised

,Unnamed: 0,added_on,confidence,description,expired_on,first_active,id,indicator,last_active,last_updated,...,review_status,severity,share_level,source_uri,status,tags,type,ds,ts,name
1,0,2016-01-01T23:29:55+0000,90,Compromised domain in AS3741. Malicious URL:mo...,NaN,NaN,875325125915484,"{'id': '875325115915485', 'indicator': 'molgri...",NaN,2016-05-21T23:43:18+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:29:55,Cloudmark Threat Exchange
2,1,2016-01-01T23:24:49+0000,90,Compromised domain in AS32613. Malicious URL:p...,NaN,NaN,953897001325079,"{'id': '953896997991746', 'indicator': 'p13855...",NaN,2016-05-26T22:17:16+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:24:49,Cloudmark Threat Exchange
3,2,2016-01-01T23:24:48+0000,90,Compromised domain in AS32613. Malicious URL:h...,NaN,NaN,942876809114340,"{'id': '942876805781007', 'indicator': 'develo...",NaN,2016-05-29T00:08:32+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:24:48,Cloudmark Threat Exchange
4,3,2016-01-01T22:56:19+0000,90,Compromised domain in AS3223. Malicious URL:ww...,NaN,NaN,912399758852107,"{'id': '912399755518774', 'indicator': 'bateri...",NaN,2016-05-28T02:14:54+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:56:19,Cloudmark Threat Exchange
5,4,2016-01-01T22:49:12+0000,90,Compromised domain in AS6724. Malicious URL:ht...,NaN,NaN,930020557087476,"{'id': '930020553754143', 'indicator': 'predif...",NaN,2016-05-29T17:06:50+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:49:12,Cloudmark Threat Exchange
6,5,2016-01-01T22:49:11+0000,90,Compromised domain in AS20013. Malicious URL:i...,NaN,NaN,824054484369537,"{'id': '824054481036204', 'indicator': 'iprote...",NaN,2016-05-31T15:06:12+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:49:11,Cloudmark Threat Exchange
7,6,2016-01-01T22:48:10+0000,90,Compromised domain in AS46606. Malicious URL:h...,NaN,NaN,983145918419311,"{'id': '983145915085978', 'indicator': 'live--...",NaN,2016-05-19T12:49:11+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:48:10,Cloudmark Threat Exchange
8,7,2016-01-01T22:02:25+0000,90,Compromised domain in AS42355. Malicious URL:h...,NaN,NaN,946343195412806,"{'id': '946343192079473', 'indicator': 'kdev.d...",NaN,2016-05-27T10:34:31+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:02:25,Cloudmark Threat Exchange
9,8,2016-01-01T21:45:08+0000,90,Compromised domain in AS48505. Malicious URL:s...,NaN,NaN,1144064518967456,"{'id': '1144064515634123', 'indicator': 'slubn...",NaN,2016-05-26T05:05:23+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 21:45:08,Cloudmark Threat Exchange
10,9,2016-01-01T21:34:56+0000,90,Compromised domain in AS132145. Malicious URL:...,NaN,NaN,946455475422671,"{'id': '946455472089338', 'indicator': 'austra...",NaN,2016-05-25T05:20:40+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 21:34:56,Cloudmark Threat Exchange


In [20]:
df = d_filtered_compromised.filter(items=['description', 'ts'])
df

,description,ts
1,Compromised domain in AS3741. Malicious URL:mo...,2016-01-01 23:29:55
2,Compromised domain in AS32613. Malicious URL:p...,2016-01-01 23:24:49
3,Compromised domain in AS32613. Malicious URL:h...,2016-01-01 23:24:48
4,Compromised domain in AS3223. Malicious URL:ww...,2016-01-01 22:56:19
5,Compromised domain in AS6724. Malicious URL:ht...,2016-01-01 22:49:12
6,Compromised domain in AS20013. Malicious URL:i...,2016-01-01 22:49:11
7,Compromised domain in AS46606. Malicious URL:h...,2016-01-01 22:48:10
8,Compromised domain in AS42355. Malicious URL:h...,2016-01-01 22:02:25
9,Compromised domain in AS48505. Malicious URL:s...,2016-01-01 21:45:08
10,Compromised domain in AS132145. Malicious URL:...,2016-01-01 21:34:56


In [34]:
ex_df = df.description.str.extract(r'(AS.\d*)')
ex_df[0].value_counts()[:20]

AS26496     10604
AS46606      4083
AS20013      2627
AS16276      2109
AS24940      1365
AS8342        809
AS8560        791
AS36351       735
AS13335       723
AS32475       687
AS27715       645
AS12824       544
AS197695      528
AS25535       525
AS33070       495
AS34619       489
AS29182       487
AS9123        473
AS16509       421
AS32392       421
Name: 0, dtype: int64

```
AS      | CC | Registry | Allocated  | AS Name
26496   | US | arin     | 2002-10-01 | AS-26496-GO-DADDY-COM-LLC - GoDaddy.com, LLC, US
46606   | US | arin     | 2008-10-24 | UNIFIEDLAYER-AS-1 - Unified Layer, US
20013   | US | arin     | 2001-03-14 | CYRUSONE - CyrusOne LLC, US
16276   | FR | ripencc  | 2001-02-15 | OVH, FR
24940   | DE | ripencc  | 2002-06-03 | HETZNER-AS, DE
8342    | RU | ripencc  | 1997-06-11 | RTCOMM-AS, RU
8560    | DE | ripencc  | 1997-11-26 | ONEANDONE-AS Brauerstrasse 48, DE
36351   | US | arin     | 2005-12-12 | SOFTLAYER - SoftLayer Technologies Inc., US
13335   | US | arin     | 2010-07-14 | CLOUDFLARENET - Cloudflare, Inc., US
32475   | US | arin     | 2005-07-21 | SINGLEHOP-LLC - SingleHop LLC, US
27715   | BR | lacnic   | 2003-08-12 | Locaweb ServiÃ§os de Internet S/A, BR
12824   | PL | ripencc  | 1999-11-26 | HOMEPL-AS, PL
197695  | RU | ripencc  | 2011-03-28 | AS-REG, RU
25535   | RU | ripencc  | 2002-12-27 | ASN-RUCENTER-HOSTING, RU
33070   | US | arin     | 2004-09-24 | RMH-14 - Rackspace Hosting, US
34619   | TR | ripencc  | 2005-02-28 | CIZGI, TR
29182   | LU | ripencc  | 2003-06-23 | ISPSYSTEM-AS, LU
9123    | RU | ripencc  | 2008-04-01 | TIMEWEB-AS, RU
16509   | US | arin     | 2000-05-04 | AMAZON-02 - Amazon.com, Inc., US
32392   | US | arin     | 2004-04-26 | OPENTRANSFER-ECOMMERCE - The Endurance International Group, Inc., US
```